This works

In [1]:
import graphix.command
from graphix.random_objects import Circuit, rand_circuit
from graphix.states import BasicStates
from graphix.sim.density_matrix import DensityMatrixBackend
from graphix.sim.statevec import StatevectorBackend, Statevec
from graphix.pauli import Pauli
from graphix.fundamentals import IXYZ
import stim
from veriphix.client import Client, Secrets, CircuitUtils
import veriphix.client
import numpy as np

/Users/sabdulsa/Codes/veriphix/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:

nqubits = 2
depth = 100
circuit = rand_circuit(nqubits, depth)
pattern = circuit.transpile().pattern 
pattern.minimize_space()

print(f"Number of nodes in the pattern : {pattern.n_node}")

## Measure output nodes, to have classical output
classical_output = pattern.output_nodes
for onode in classical_output:
    pattern.add(graphix.command.M(node=onode))



secrets = Secrets(r=True, a=True, theta=True)
client = Client(pattern=pattern, secrets=secrets)

n_good_output = 0

for _i in range(1):
    backend = DensityMatrixBackend()
    client.delegate_pattern(backend=backend)

Number of nodes in the pattern : 870
